### Importing Libraries

In [29]:
import pymongo
from pymongo import MongoClient
import json
import folium
from bs4 import BeautifulSoup
import requests
import pandas as pd

client = MongoClient("mongodb://localhost:27017")


db = client['DAP_Project']
collection = db['LAPD_Districts']

In [8]:
link = requests.get("https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles")
soup = BeautifulSoup(link.text, "lxml")

sections=soup.find_all(class_="div-col")
#sections=soup.find_all("p")
places = BeautifulSoup(str(sections)).find_all('li')
#print(sections)

neighborhoods_list = []

for div in places:
    if div.find('a').contents[0] == '[40]':
        neighborhoods_list.append('Pico Robertson')
    else:
        neighborhoods_list.append(div.find('a').contents[0])

        
#len(neighborhoods_list)
#print(neighborhoods_list)

In [9]:
def getNeighborhoodData(neighborhoods_list):
    geoKey = 'your-geocodingAPI-key'
    try:
        with open("LA_Neighborhoods.json") as data:
            jsonList = json.load(data)
    except IOError:
        jsonList = []
        for neighborhood in neighborhoods_list:
            parameters = {
                "address": "%s, Los Angeles, CA" % neighborhood,
                "key": geoKey 
            }
            results = requests.get(
                'https://maps.googleapis.com/maps/api/geocode/json', 
                params=parameters
            ).json()
            jsonList.append(results)
        with open("LA_Neighborhoods.json", 'w') as outputFile:
            json.dump(jsonList, outputFile)
        
    return jsonList

In [10]:
jsonList = getNeighborhoodData(neighborhoods_list)
print(jsonList)

[{'results': [{'address_components': [{'long_name': 'Angelino Heights', 'short_name': 'Angelino Heights', 'types': ['neighborhood', 'political']}, {'long_name': 'Los Angeles', 'short_name': 'Los Angeles', 'types': ['locality', 'political']}, {'long_name': 'Los Angeles County', 'short_name': 'Los Angeles County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '90026', 'short_name': '90026', 'types': ['postal_code']}], 'formatted_address': 'Angelino Heights, Los Angeles, CA 90026, USA', 'geometry': {'location': {'lat': 34.0702889, 'lng': -118.2547965}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 34.0795311, 'lng': -118.2387891}, 'southwest': {'lat': 34.0610457, 'lng': -118.2708039}}}, 'place_id': 'ChIJY1MciAHHwoARZdkFYSGvw7o', 'types': ['neighborhood', 'po

In [11]:
neighborData = []
for element in jsonList:
    if element['results']:
        neighborData.append([
            element['results'][0]['address_components'][0]['long_name'],
            element['results'][0]['geometry']['location']['lat'],
            element['results'][0]['geometry']['location']['lng']
        ])
print(neighborData)

[['Angelino Heights', 34.0702889, -118.2547965], ['2611', 33.9955625, -118.3219766], ['Angelus Avenue', 34.0854982, -118.2684407], ['Arleta', 34.2504595, -118.4338345], ['Arlington Heights', 34.0422222, -118.3188889], ['Arts District', 34.0418947, -118.2326448], ['Atwater Village', 34.1172904, -118.2614332], ['Baldwin Hills', 34.006677, -118.3505779], ['3650', 34.0107541, -118.3370033], ['Baldwin Village', 34.0150915, -118.3476559], ['Baldwin Vista', 34.0134562, -118.3627367], ['2679-2687', 34.1196957, -118.3210548], ['Bel Air', 34.1002455, -118.459463], ['Benedict Canyon', 34.0939284, -118.4324553], ['Beverly Crest', 34.1012952, -118.4162542], ['Beverly Glen', 34.1077162, -118.442596], ['9663', 34.0691217, -118.4067693], ['North Beverly Park', 34.1177924, -118.4177707], ['Beverlywood', 34.0494132, -118.3952319], ['Boyle Heights', 34.0297895, -118.2117257], ['Brentwood', 34.0521011, -118.4732464], ['Los Angeles', 34.0717832, -118.4710675], ['Brentwood Glen', 34.0655045, -118.4626788], 

### Converting to pandas DataFrame

In [12]:
laDF = pd.DataFrame(
    data=neighborData,
    columns=["Neighborhood", "Latitude", "Longitude"],
)

laDF.head(10)
data = laDF.to_dict(orient='records')
print(data)
#run the below line of code only once, otherewise it will make entry of duplicate records
collection.insert_many(data)


[{'Neighborhood': 'Angelino Heights', 'Latitude': 34.0702889, 'Longitude': -118.2547965}, {'Neighborhood': '2611', 'Latitude': 33.9955625, 'Longitude': -118.3219766}, {'Neighborhood': 'Angelus Avenue', 'Latitude': 34.0854982, 'Longitude': -118.2684407}, {'Neighborhood': 'Arleta', 'Latitude': 34.2504595, 'Longitude': -118.4338345}, {'Neighborhood': 'Arlington Heights', 'Latitude': 34.0422222, 'Longitude': -118.3188889}, {'Neighborhood': 'Arts District', 'Latitude': 34.0418947, 'Longitude': -118.2326448}, {'Neighborhood': 'Atwater Village', 'Latitude': 34.1172904, 'Longitude': -118.2614332}, {'Neighborhood': 'Baldwin Hills', 'Latitude': 34.006677, 'Longitude': -118.3505779}, {'Neighborhood': '3650', 'Latitude': 34.0107541, 'Longitude': -118.3370033}, {'Neighborhood': 'Baldwin Village', 'Latitude': 34.0150915, 'Longitude': -118.3476559}, {'Neighborhood': 'Baldwin Vista', 'Latitude': 34.0134562, 'Longitude': -118.3627367}, {'Neighborhood': '2679-2687', 'Latitude': 34.1196957, 'Longitude': 

### As you can see there are some neighborhoods which has there address saved in the neighborhood column. So we are doing it manually

In [13]:
#correct anomalies
laDF.loc[1, "Neighborhood"] = "Angeles Mesa"
laDF.loc[8, "Neighborhood"] = "Baldwin Hills Crenshaw"
laDF.loc[11, "Neighborhood"] = "Beachwood Canyon"
laDF.loc[16, "Neighborhood"] = "Beverly Grove"
laDF.loc[33, "Neighborhood"] = "Chesterfield Square"
laDF.loc[43, "Neighborhood"] = "East Gate Bell Air"
laDF.loc[59, "Neighborhood"] = "Flower District"
laDF.loc[61, "Neighborhood"] = "Gallery Row"
laDF.loc[83, "Neighborhood"] = "Jewelry District"
laDF.loc[96, "Neighborhood"] = "Little Italy"
laDF.loc[118, "Neighborhood"] = "Old Bank District"
laDF.loc[124, "Neighborhood"] = "Park La Brea"
laDF.loc[144, "Neighborhood"] = "Sonoratown"
laDF.loc[184, "Neighborhood"] = "Westside Village"

In [14]:
laDF.dtypes

Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [15]:
  laDF.head(10)

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.995562,-118.321977
2,Angelus Avenue,34.085498,-118.268441
3,Arleta,34.250459,-118.433835
4,Arlington Heights,34.042222,-118.318889
5,Arts District,34.041895,-118.232645
6,Atwater Village,34.117290,-118.261433
7,Baldwin Hills,34.006677,-118.350578
8,Baldwin Hills Crenshaw,34.010754,-118.337003
9,Baldwin Village,34.015091,-118.347656


In [16]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [17]:
from geopy.geocoders import Nominatim

address = 'Los Angeles'

geolocator = Nominatim(user_agent = "ExploreLA")
LA_location = geolocator.geocode(address)
LA_latitude = LA_location.latitude
LA_longitude = LA_location.longitude

print('The geograpical coordinates of Los Angeles are {}, {}.'.format(LA_latitude, LA_longitude))

The geograpical coordinates of Los Angeles are 34.0536909, -118.242766.


### Plotting neighbourhoods on the MaP of LA 

In [18]:
mapLA = folium.Map(
    location=[LA_latitude, LA_longitude], 
    tiles='Stamen Toner', 
    zoom_start=10, 
)

# add markers to map
for lat, lng, neighborhood in zip(laDF['Latitude'], laDF['Longitude'], laDF['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.Marker(
        [lat, lng],
        popup = label,
    ).add_to(mapLA)
   

In [19]:
#This comment is added by Ashwyn as a Test


In [20]:
mapLA

In [27]:
pip install pymongo

In [30]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['DAP_Project']
collection = db['LAPD_Division']

In [32]:
with open('LAPD_Divisions.json') as file:
          file_data = json.load(file)
          print(file_data)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'kind': 'LAPD Division', 'external_id': '12', 'name': '77th Street', 'slug': '77th-street-lapd-division', 'set': '/1.0/boundary-set/lapd-divisions/', 'metadata': {'name': '77th Street', 'url': 'http://www.lapdonline.org/77th_street_community_police_station', 'number': '12', 'phone_numb': '213-485-4164', 'address': '7600 S. Broadway', 'square_mil': 11.3334694611}, 'resource_uri': '/1.0/boundary/77th-street-lapd-division/'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-118.31489489515535, 34.00371678060566], [-118.2810618703952, 34.00378143593843], [-118.28102528408968, 34.00134137250015], [-118.28046807812566, 33.99488115273315], [-118.28050467938607, 33.98322504640739], [-118.28079549210042, 33.97984943789391], [-118.28079188121573, 33.97467729218511], [-118.25626692365674, 33.974772374122175], [-118.25626016431879, 33.96017857530402], [-118.29166512945436, 33.96003367665656], [-118.29167168624657

In [33]:
collection.insert_one(file_data)